# Imports

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import seaborn as sns
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = 'notebook'

from scipy.signal import find_peaks
from scipy.signal import savgol_filter

from functions.read_data import read_history

# graph font settings
rcParams['font.family'] = "serif"     
rcParams['font.size']=12

generations = 500
tau_values = 100

# Load in simulation data

In [ ]:
# Store data in 100 lists, each with 500 dicts, corresponding to tau and generations respectively
history_csv_file = '../data/history_data.csv'
history_array = []
history_array = read_history(history_array, history_csv_file)

In [ ]:
# Store important values
Nc_values_3d = np.zeros((generations, tau_values))
Nc_values_3d_smoothed = np.zeros((generations, tau_values))
Nc_derivative_3d = np.zeros((generations, tau_values))
Nc_derivative_3d_smoothed = np.zeros((generations, tau_values))

# Populate the Nc values and their derivatives for the 3D plot
for i, tau in enumerate(range(tau_values)):
    for j, generation in enumerate(range(generations)):
        Nc_values_3d[j, i] = history_array[tau][generation]['Nc']
    # Compute the derivative of Nc with respect to generations
    Nc_derivative_3d[:, i] = np.gradient(Nc_values_3d[:, i].astype(int))

In [ ]:
# Parameters for Savitzky-Golay filter
window_length = 51  # Size of the filtering window
polyorder = 5     # The order of the polynomial used to fit the samples

# Apply the Savitzky-Golay filter to each curve
for i in range(Nc_values_3d.shape[1]):
    Nc_values_3d_smoothed[:, i] = savgol_filter(Nc_values_3d[:, i], window_length, polyorder)
    Nc_derivative_3d_smoothed[:, i] = savgol_filter(Nc_derivative_3d[:, i], window_length, polyorder)

### Method 1: Plot critical points of f(x) = Nc vs Generations against Bifurcation Parameter 'Tau'

In [ ]:
critical_points_by_tau = []
taus = []
final_value = []

for tau in range(0, 100):
    Nc_data = np.array([history_array[tau][i]['Nc'] for i in range(generations)]).astype(int)
    
    peaks, _ = find_peaks(Nc_data, prominence=1)
    peaks_values = Nc_data[peaks]
    troughs, _ = find_peaks(-Nc_data, prominence=1)
    troughs_values = Nc_data[troughs]
    final_value.append(Nc_data[-1])
    combined_values = np.concatenate((peaks_values, troughs_values))
    critical_points_by_tau.append(combined_values)
    taus.append(tau)

for tau in range(0, 100):
    y_values = critical_points_by_tau[tau]
    if y_values.size == 0:
        y_values = [final_value[tau]]
    x_values = [tau] * len(y_values)
    
    plt.scatter(x_values, y_values, s = 1)
    
plt.title('Bifurcation Plot')
plt.xlabel('Tau')
plt.ylabel('Critical Points')
plt.show()

### Method 2: Plot phase-space (using derivative) against tau

In [ ]:
fig = plt.figure(figsize=(15, 11), dpi=300)
ax = fig.add_subplot(111, projection='3d')

# Plot the smoothed data
for i, tau in enumerate(range(0,tau_values)):
    ax.plot([tau] * generations, Nc_values_3d_smoothed[:, tau], Nc_derivative_3d_smoothed[:, tau], label=f'Tau = {tau}')

# Set labels for the axes
ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$N_c$')
ax.set_zlabel(r'$\frac{dN_c}{dGenerations}$')
ax.xaxis.labelpad=25
ax.yaxis.labelpad=25
ax.zaxis.labelpad=25
ax.set_title(r'Phase space plot of $N_C$ for Various Values of $\tau$')

# Show the 3D plot
plt.show()

In [ ]:
rcParams['font.size']=20

fig = plt.figure(figsize=(15, 11), dpi=300)
ax = fig.add_subplot(111, projection='3d')

colormap = sns.color_palette('husl', 6)

# Plot a selected range
selected_range = list(range(0,30,5))
for i, tau in enumerate(selected_range):
    for j in range(generations-1):
        ax.plot([tau]*2, Nc_values_3d_smoothed[:, tau][j:j+2], Nc_derivative_3d_smoothed[:, tau][j:j+2], color=colormap[i],alpha=(0.1+(0.9*float(j)/(generations))), label=f'Tau = {tau}')

# Set labels for the axes
ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$N_c$')
ax.set_zlabel(r'$\frac{dN_c}{dGenerations}$')
ax.xaxis.labelpad=25
ax.yaxis.labelpad=25
ax.zaxis.labelpad=25
ax.set_title(r'Phase space plot of $N_C$ for Various Values of $\tau$')

In [ ]:
rcParams['font.size']=20
fig = plt.figure(figsize=(15, 11), dpi=300)
ax = fig.add_subplot(111, projection='3d')

# Plot a selected range
selected_range = list(range(30,40,2))
for i, tau in enumerate(selected_range):
    for j in range(generation-1):
        ax.plot([tau] * 2, Nc_values_3d_smoothed[:, tau][j:j+2], Nc_derivative_3d_smoothed[:, tau][j:j+2],color=colormap[i],alpha=(0.4+(0.6*float(j)/(generations))), label=f'Tau = {tau}')

# Set labels for the axes
ax.set_xticklabels([30,32,34,36,38])
ax.set_xticks([30,32,34,36,38])
ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$N_c$')
ax.set_zlabel(r'$\frac{dN_c}{dGenerations}$')
ax.xaxis.labelpad=25
ax.yaxis.labelpad=25
ax.zaxis.labelpad=25
ax.set_title(r'Phase space plot of $N_c$ for Various Values of $\tau$')


### Interactive plot for further analysis

In [ ]:
# Create traces for each tau value
traces = []
for i, tau in enumerate(range(tau_values)):
    trace = go.Scatter3d(
        x=[tau] * generations,       # Tau values along the x-axis
        y=Nc_values_3d[:, i],        # Nc values along the y-axis
        z=Nc_derivative_3d[:, i],    # Derivative values along the z-axis
        mode='lines',
        name=f'Tau = {tau}'
    )
    traces.append(trace)

# Define layout
layout = go.Layout(
    title='3D Plot of Nc and its Derivative over Generations',
    scene=dict(
        xaxis_title='Tau',
        yaxis_title='Nc (Number of Cancer Cells)',
        zaxis_title='dNc/dGenerations'
    ),
    margin=dict(l=0, r=0, b=0, t=0)  # Tight layout
)

fig = go.Figure(data=traces, layout=layout)
fig.show()


In [ ]:
plt.figure(figsize=(14, 8))

# Plot Nc for the selected tau values
for tau in range(33,39):
    Nc_values = np.array([history_array[tau][g]['Nc'] for g in range(generations)]).astype(int)
    plt.plot(range(generations), Nc_values, label=f'Tau = {tau}')

plt.xlabel('Generations')
plt.ylabel('Nc (Number of Cancer Cells)')
plt.title('Nc over Generations for Various Values of Tau')
plt.legend()
plt.show()


### Method 3: Use time delay embedding instead of derivative

In [ ]:
# Store important values
Nc_values_3d = np.zeros((generations, tau_values))

# Populate the Nc values for the 3D plot
for i, tau in enumerate(range(tau_values)):
    for j, generation in enumerate(range(generations)):
        Nc_values_3d[j, i] = history_array[tau][generation]['Nc']

# Choose embedding dimension (2 for a 2D phase space plot)
embedding_dim = 2
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Create phase space plots for each tau
for tau in range(80, tau_values):
    # Ensure there's enough data for the given tau
    if tau < generations - 1:
        # Reconstruct the phase space for this tau
        time_series = Nc_values_3d[:, tau]

        # Adjust the slicing to ensure equal length arrays
        if len(time_series) > 2 * tau:
            phase_space = np.array([time_series[tau:-tau], time_series[2*tau:]])
            phase_space = phase_space.T

            # Creating an array for tau to match the dimensions of the phase space points
            tau_array = np.full(len(phase_space), tau)

            ax.plot(tau_array, phase_space[:, 0], phase_space[:, 1], '.-')

ax.set_title('3D Phase Space Plot with Tau on x-axis')
ax.set_xlabel('Tau')
ax.set_ylabel('Nc(t)')
ax.set_zlabel('Nc(t + tau)')
plt.show()
